# 수집된 뉴스 기사 및 댓글에 대한 감정 분석
## * Doc2Vec 
* 데이터 
> 2017년 12월 1일부터 2018년 2월 1일까지 63일간 [네이버](http://www.naver.com)와 [다음](http://www.daum.net)의 랭킹뉴스와 뉴스의 댓글을 크롤링함.

In [1]:
import pickle
import html
import multiprocessing
from collections import namedtuple, OrderedDict
import re
import sys
import os
from glob import glob
import warnings

os.environ['KERAS_BACKEND']='tensorflow'

from numba import jit
import numpy as np
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
import pandas as pd

from gensim.models import doc2vec, KeyedVectors
from gensim.models.doc2vec import TaggedDocument

from konlpy.utils import pprint

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report, roc_curve,  accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import scale, MinMaxScaler
from sklearn.manifold import TSNE

import keras.backend.tensorflow_backend as K
from keras.preprocessing import sequence
from keras_tqdm import TQDMCallback, TQDMNotebookCallback
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.layers import Input, Flatten, Dense, Embedding, embeddings, merge, Dropout, Activation,  LSTM, Bidirectional, SimpleRNN, GRU
from keras.layers.convolutional import Conv1D, Conv2D
from keras.layers.pooling import MaxPooling1D, GlobalMaxPooling1D
from keras.layers.normalization import BatchNormalization
from keras.layers.core import SpatialDropout1D
from keras.utils import np_utils
from tensorflow.python.client import device_lib
from keras.layers.merge import dot

import xgboost as xgb

import matplotlib.pyplot as plt

import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.plotting import figure, show, output_notebook

/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import Database_Handler as dh
import Basic_Module as bm

In [3]:
from ckonlpy.tag import Twitter
from konlpy.tag import Mecab
ct = Twitter()
mecab = Mecab()

## Stopwords

In [4]:
stopwords = open('./data/stopwordsList.txt',encoding='utf-8').readlines()
stopwords = list(map(lambda x: x.strip(), stopwords))

## Load Data

In [5]:
#Naver
naverData = pickle.load(open('./data/pre_data/stastics/for_statistics_Naver_from_mongodb.pickled','rb'))
naverData = pd.DataFrame.from_dict(naverData, orient = 'index')
naverData.reset_index(inplace = True)
naverData.rename(columns = {'index' : 'id'}, inplace = True)
#Daum
daumData = pickle.load(open('./data/pre_data/stastics/for_statistics_daum_from_mongodb.pickled','rb'))
daumData = pd.DataFrame.from_dict(daumData, orient = 'index')
daumData.reset_index(inplace = True)
daumData.rename(columns = {'index' : 'id'}, inplace = True)

print ('Naver : {}'.format(naverData.shape))
print ('Daum : {}'.format(daumData.shape))

Naver : (15120, 11)
Daum : (9372, 11)


In [15]:
extNaverData = naverData.loc[:, ['id', 'title', 'date', 'press', 'rank', 'category', 'number_of_comment', 'number_of_crawled_comment']].copy()
extNaverData['site'] = pd.Series(['Naver']*extNaverData.shape[0])
extDaumData = daumData.loc[:, ['id', 'title', 'date', 'press', 'rank', 'category', 'number_of_comment', 'number_of_crawled_comment']].copy()
extDaumData['site'] = pd.Series(['daum']*extDaumData.shape[0])

In [32]:
if sys.platform =='darwin':
    loadModelPath = '/Volumes/disk1/model/'
    classifierPath = '/Volumes/disk1/data/pre_data/classifier/'
    #newsPath = '/Volumes/data/pre_data/news_sentiment/'
    newsPath = './data/pre_data/news_sentiment/'
elif sys.platform =='win32':
    loadModelPath = 'd:/model/'
    classifierPath = 'd:/data/pre_data/classifier/'
    newsPath = './data/pre_data/news_sentiment/'

## TaggedDocument For Doc2Vec

In [8]:
TaggedDocument = namedtuple('TaggedDocument', 'words tags sentiment')

### Comment

In [11]:
mongodb = dh.ToMongoDB(*dh.AWS_MongoDB_Information())
dbname = 'hy_db'
useDB = dh.Use_Database(mongodb, dbname)
useCollection = dh.Use_Collection(useDB, 'comments')

## Doc2Vec Model

### Twitter

In [9]:
taggerName = 'ct'

In [43]:
for rowIdx in extNaverData.index:
    data = extNaverData.loc[rowIdx]
    commentsForNews = useCollection.find({'site' : data['site'],
                      'category' : data['category'], 'date' : data['date'], 'rank' : int(data['rank'])})
    realNumCount = commentsForNews.count()
    print ('Title : {}'.format(data['title']))
    print ('Number of crawled comments for this news : {}'.format(realNumCount))
    print ('Number of comments : {}'.format(data['number_of_comment']))
    print ('Number of comments recorded in DB : {}'.format(data['number_of_crawled_comment']))
    break

Title : 北외무성 "전쟁 바라지 않지만 결코 피하지 않을 것"
Number of crawled comments for this news : 1465
Number of comments : 1713
Number of comments recorded in DB : 1474
